Install dependencies

In [ ]:
!pip install google-generativeai requests python-dotenv

Load environment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

Initialize Gemini

In [ ]:
import google.generativeai as genai

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

model = genai.GenerativeModel("gemini-2.5-flash")

Quick Gemini test

In [ ]:
response = model.generate_content("Hello there")
print(response.text)

GitHub file extractor

In [ ]:
import requests
import base64

def get_default_branch(repo):
    url = f"https://api.github.com/repos/{repo}"
    r = requests.get(url)
    r.raise_for_status()
    return r.json()["default_branch"]

def get_github_file(repo, path, branch=None):
    if branch is None:
        branch = get_default_branch(repo)

    url = f"https://api.github.com/repos/{repo}/contents/{path}?ref={branch}"
    r = requests.get(url)

    if r.status_code == 404:
        raise Exception(f"File not found: {repo}/{path} on branch {branch}")

    r.raise_for_status()
    data = r.json()
    return base64.b64decode(data["content"]).decode("utf-8")



Test GitHub extraction

In [ ]:
code = get_github_file("psf/requests", "src/requests/__init__.py")
print(code[:500])

Decide the language

In [ ]:
def detect_language(path):
    if path.endswith(".py"):
        return "python"
    if path.endswith(".ts"):
        return "typescript"
    if path.endswith(".js"):
        return "javascript"
    return "unknown"

Prompt for documentation generation

In [ ]:
def add_documentation(code, language):
    prompt = f"""
You are a senior {language} developer.

Task:
Add clear, professional documentation to the following source code.

Rules:
- Do NOT change logic
- Add docstrings / comments where appropriate
- Keep formatting clean
- Follow best practices for {language}

Source code:
{code}
"""
    response = model.generate_content(prompt)
    return response.text


Prompt for test generation

In [ ]:
def generate_tests(code, language):
    prompt = f"""
You are a senior {language} developer.

Task:
Write unit tests for the following source code.

Rules:
- Use best testing framework for {language}
- Cover main logic and edge cases
- Return ONLY test code
- Do not explain anything

Source code:
{code}
"""
    response = model.generate_content(prompt)
    return response.text

Full agent pipeline

In [ ]:
def document_and_test(repo, path):
    code = get_github_file(repo, path)
    language = detect_language(path)

    documented_code = add_documentation(code, language)
    tests = generate_tests(code, language)

    return {
        "original": code,
        "documented": documented_code,
        "tests": tests
    }

agent testing

In [ ]:
result = document_and_test(
    "pallets/flask",
    "src/flask/cli.py"
)

print("===== DOCUMENTED CODE =====")
print(result["documented"])

print("\n===== GENERATED TESTS =====")
print(result["tests"])


Save results to files

In [ ]:
def save_output(path, documented_code, tests):
    base = path.split("/")[-1].replace(".py", "")
    
    with open(f"{base}_documented.py", "w", encoding="utf-8") as f:
        f.write(documented_code)

    with open(f"test_{base}.py", "w", encoding="utf-8") as f:
        f.write(tests)
        
save_output(
    "cli.py",
    result["documented"],
    result["tests"]
)